In [2]:
import librosa
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
warnings.filterwarnings('ignore')
import pathlib


# New Section

In [4]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
train = ' ESM ESMm MDR Murmurs1 Murmurs Others PSM PSMm diastolic systolic '.split()
for t in train:
    pathlib.Path(f'img_data/{t}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir (f'/content/drive/MyDrive/Mysteth/{t}'):
        test_data = f'/content/drive/MyDrive/Mysteth/{t}/{filename}'
        y,sr = librosa.load(test_data, mono=True)
        plt.specgram(y, NFFT=256, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{t}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 800x800 with 0 Axes>

In [5]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1,5):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [7]:
file = open('RSH.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for t in train :
    for filename in os.listdir(f'/content/drive/MyDrive/Mysteth/{t}'):
        test_data = f'/content/drive/MyDrive/Mysteth/{t}/{filename}'
        y, sr = librosa.load(test_data, mono=True, duration=30)
        rmse = librosa.feature.rms(y=y)[0]
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {t}'
        file = open('RSH.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())